In [3]:
import numpy as np
import pandas as pd
from roboflow import Roboflow
import os

In [5]:
rf = Roboflow(api_key="PYZngL70ijcagLdXpb6l")
project = rf.workspace("material-identification").project("garbage-classification-3")
dataset = project.version(2).download("tensorflow")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to GARBAGE-CLASSIFICATION-3-2 in tensorflow:: 100%|██████████| 10472/10472 [00:01<00:00, 8166.34it/s]


In [28]:
train = pd.read_csv('GARBAGE-CLASSIFICATION-3-2/train/_annotations.csv')
test = pd.read_csv('GARBAGE-CLASSIFICATION-3-2/test/_annotations.csv')
val = pd.read_csv('GARBAGE-CLASSIFICATION-3-2/valid/_annotations.csv')
# classes = ['PAPER', 'GLASS', 'METAL_CARDBOARD_PLASTIC','BIODEGRADABLE' ,'trash']
combined_data = pd.concat([train, test, val], ignore_index=True)
print(len(train.index),len(test.index),len(val.index))
print(len(combined_data.index))

51611 3563 18916
74090
